<a href="https://colab.research.google.com/github/SergeiShilkoysky/BigData/blob/main/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задача: сделайте mapper и reducer, чтобы посчитать среднее и дисперсию оценок за фильм.

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
import pandas as pd
import time
from pathlib import Path

In [ ]:
import json
from functools import reduce

In [ ]:
dataset_path = Path('imdb-user-reviews', 'song_lyrics.csv')
if not dataset_path.is_file():
    od.download('https://www.kaggle.com/datasets/sadmadlad/imdb-user-reviews')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: sergeishi
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/sadmadlad/imdb-user-reviews


100%|██████████| 15.1M/15.1M [00:00<00:00, 56.4MB/s]


In [ ]:
!cat imdb-user-reviews/

{"title": "Joker", "movieIMDbRating": 8.4, "totalRatingCount": 1213550, "totalUserReviews": "11.3K", "totalCriticReviews": "697", "metaScore": "59", "movieGenres": ["Crime", "Drama", "Thriller"], "directors": ["Todd Phillips"], "datePublished": "2019-10-04", "creators": ["Todd Phillips", "Scott Silver", "Bob Kane"], "mainStars": ["Joaquin Phoenix", "Robert De Niro", "Zazie Beetz"], "description": "A mentally troubled stand-up comedian embarks on a downward spiral that leads to the creation of an iconic villain.", "duration": 122}

In [ ]:
def calc_mean_variance_movieScores():
  n, mean, M2 = 0, 0.0, 0
  for path in Path('imdb-user-reviews').glob('**/*'):
      if path.is_file() and path.suffix == '.json':
          with open(path, 'r') as f:
              info = json.load(f)
          score = float(info['movieIMDbRating'])
          n += 1
          delta = score - mean
          mean += delta / n
          M2 += delta * (score - mean)

  print(mean, (M2 / n) ** (1/2))

In [ ]:
def mapper(path):
    if path.is_file() and path.suffix == '.json':
      with open(path, 'r') as f:
        info = json.load(f)
      return (float(info['movieIMDbRating']), )


def reducer(score_data1, score_data2):
    if score_data1 is None and score_data2 is None:
      return None
    elif score_data1 is None:
      return score_data2
    elif score_data2 is None:
      return score_data1
    else:
      scores = []
      if len(score_data1) == 1:
        n, mean, M2 = 0, 0.0, 0
        scores.append(score_data1[0])
      else:
        n, mean, M2 = score_data1
      scores.append(score_data2[0])
      for score in scores:
        n += 1
        delta = score - mean
        mean += delta / n
        M2 += delta * (score - mean)
      return n, mean, M2

In [ ]:
# @title Параллельная обработка с использованием map и reduce:
%%time
n, mean, M2 = reduce(reducer, map(mapper, Path('imdb-user-reviews').glob('**/*')))
print(mean, (M2 / n) ** (1/2))

8.030000000000001 1.0517128885774862
CPU times: user 473 µs, sys: 4.22 ms, total: 4.69 ms
Wall time: 4.77 ms


Соберем mapper и reducer:

In [ ]:
# @title Синхронный подход:
%%time
calc_mean_variance_movieScores()

8.030000000000001 1.0517128885774862
CPU times: user 1.34 ms, sys: 991 µs, total: 2.33 ms
Wall time: 2.33 ms


In [ ]:
# @title Параллельная обработка с использованием map и reduce:
%%time
n, mean, M2 = reduce(reducer, map(mapper, Path('imdb-user-reviews').glob('**/*')))
print(mean, (M2 / n) ** (1/2))

8.030000000000001 1.0517128885774862
CPU times: user 2.94 ms, sys: 623 µs, total: 3.56 ms
Wall time: 3.36 ms


In [ ]:
from joblib import Parallel, delayed

In [ ]:
# @title Параллельная обработка с помощью joblib:
%%time
n, mean, M2 = reduce(reducer, Parallel(n_jobs=2)(delayed(mapper)(path) for path in Path ('imdb-user-reviews').glob('**/*')))
print(mean, (M2 / n) ** (1/2))

8.030000000000001 1.0517128885774862
CPU times: user 13.1 ms, sys: 1.9 ms, total: 15 ms
Wall time: 25.6 ms
